# Asynchronous Shopping Assistant using Strands

## Overview

In this tutorial, we'll build an asynchronous shopping assistant that can search for products on Amazon while maintaining a conversation with the user. This use case demonstrates how to leverage Amazon Bedrock AgentCore's asynchronous capabilities with Strands to create a responsive agent that can perform time-consuming tasks in the background.

### Use Case Details

| Information         | Details                                                                      |
|---------------------|------------------------------------------------------------------------------|
| Use case type       | Shopping Assistant                                                           |
| Agent type          | Asynchronous                                                                 |
| Agentic Framework   | Strands                                                                      |
| LLM model           | Anthropic Claude 3 Sonnet & Haiku                                            |
| Components          | AgentCore Runtime, Browser Tool, Async Tasks                                 |
| Example complexity  | Intermediate                                                                 |
| SDK used            | Amazon BedrockAgentCore Python SDK                                           |

### Use Case Architecture

This shopping assistant demonstrates a powerful pattern for building responsive AI agents that can handle time-consuming tasks without blocking the conversation flow:

1. The user asks about products they're interested in
2. The agent launches a background task using AgentCore's async capabilities
3. While the background task runs, the agent can continue conversing with the user
4. When results are ready, they're stored in a file that the agent can access
5. The agent can retrieve and present the results when appropriate

The architecture combines several key components:
- **Strands**: For building the agent's workflow and multi-agent orchestration
- **AgentCore Runtime**: For hosting and scaling the agent
- **AgentCore Async Tasks**: For managing background operations
- **Browser Tool**: For searching and extracting product information
- **File System**: For storing and retrieving search results

![](images/architecture_diagram.png)

### Agent Flow Diagram

The following diagram illustrates the flow of our multi-agent system using Strands:

![](images/async_shopping_main.png)

#### Agent Workflow Steps

1. **User Request**: The user makes a shopping-related query (e.g., "What's the price of Echo Dot?")
2. **Fronting Agent**: Evaluates the request and determines if it requires product search
3. **Conditional Routing**: If shopping is needed, the request is routed to the Shopping Agent
4. **Background Task**: The Shopping Agent launches a background browser task using NovaAct
5. **Parallel Conversation**: While the search runs, the Fronting Agent continues conversing with the user
6. **Result Storage**: When the browser task completes, results are saved to a file
7. **Task Completion**: The background task is marked as complete in AgentCore
8. **Result Retrieval**: When the user asks for results, the Reporting Agent reads the file and presents findings
9. **Multi-Search Support**: For product comparisons, multiple browser sessions can run in parallel

### Key Features

* **Asynchronous Task Management**: Launch background tasks that don't block the conversation
* **Browser Tool Integration**: Use AgentCore's Browser Tool to search and extract product information
* **Multi-Agent Orchestration**: Coordinate between fronting, shopping, and reporting agents using Strands
* **File System Integration**: Store and retrieve search results using file system tools
* **Parallel Processing**: Handle multiple product searches simultaneously


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials with appropriate permissions
* Amazon Bedrock AgentCore SDK
* Strands
* Docker running (for local testing and deployment)

In [ ]:
# Install required packages
!pip install -r requirements.txt

## Understanding Asynchronous Agents

Traditional conversational agents process user requests synchronously, which means they block the conversation until a task is complete. This approach works well for quick operations but creates a poor user experience for time-consuming tasks like web browsing or complex searches.

Asynchronous agents solve this problem by:

1. **Non-blocking operations**: Tasks run in the background while the conversation continues
2. **Task management**: The agent can start, monitor, and retrieve results from background tasks
3. **Improved responsiveness**: Users get immediate feedback and can continue interacting

Amazon Bedrock AgentCore provides built-in support for asynchronous operations through its task management capabilities, which we'll leverage in this shopping assistant.

### The Shopping Assistant Use Case

Our shopping assistant will:
- Accept product search queries from users
- Launch background tasks to search Amazon.com using the Browser Tool
- Continue conversing with the user while searches run
- Store search results in files when complete
- Retrieve and present results when the user asks for them

This pattern is particularly valuable for e-commerce applications where product searches can take time, but users expect responsive interactions.

## Understanding the Asynchronous Shopping Assistant Code

Let's examine the key components of our asynchronous shopping assistant implementation. The code below implements a shopping assistant that can search for products on Amazon while maintaining a conversation with the user.

**Key Components:**

1. **AgentCore Integration**: Initializes the BedrockAgentCoreApp that hosts our agent
2. **Browser Tool Implementation**: Uses NovaAct to control a browser for product searches
3. **Asynchronous Task Management**: Manages background tasks that don't block the conversation
4. **Multi-Agent Architecture**: Uses three specialized agents coordinated through Strands
5. **Entrypoint Function**: Handles agent invocation with proper routing

Below is the complete implementation of our asynchronous shopping assistant:

In [ ]:
# Display the entrypoint code with syntax highlighting
!pygmentize async_shopping_with_strands.py

### Understanding the Entrypoint File

The `async_shopping_with_strands.py` file contains the complete implementation of our asynchronous shopping assistant. Let's explore its key components:

1. **AgentCore Integration**:
   ```python
   from bedrock_agentcore.runtime import BedrockAgentCoreApp
   app = BedrockAgentCoreApp()
   ```
   This initializes the AgentCore application that will host our agent.

2. **Browser Tool Implementation**:
   ```python
   @tool(name="background_shopping", description="Based on the incoming shopping request, this agent-as-a-tool starts a background shopping task and writes a result file when done.")
   def call_browser_tool(request: str):
       # ...
       thread = threading.Thread(
           target=_run_browser_task,
           args=(request,),
           daemon=True,
       )
       thread.start()
   ```
   This function runs a browser task in the background using NovaAct and AgentCore's async task management.

3. **Multi-Agent Architecture**:
   ```python
   fronting_agent = Agent(name="fronting_assistant", 
                       system_prompt="...",
                       tools=[get_tasks_info, file_read],
                       model=sonnet)
   
   shopping_agent = Agent(name="shopping_assistant", 
                       system_prompt="...",
                       tools=[call_browser_tool],
                       model=sonnet)
   
   reporting_agent = Agent(name="reporting_assistant", 
                       system_prompt="...",
                       tools=[file_read, get_tasks_info, shell, file_write],
                       model=haiku)
   ```
   This creates three specialized agents with different responsibilities in the workflow.

4. **Strands Graph Builder**:
   ```python
   builder = GraphBuilder()
   
   builder.add_node(fronting_agent, "start")
   builder.add_node(shopping_agent, "shop")
   builder.add_node(reporting_agent, "report")
   
   builder.add_edge("start", "shop", condition=only_if_shopping_needed)
   builder.add_edge("start", "report", condition=only_if_background_task_is_done)
   builder.set_entry_point("start")
   
   graph = builder.build()
   ```
   This creates a Strands graph with conditional routing between agents.

5. **Entrypoint Function**:
   ```python
   @app.entrypoint
   def handler(payload, context):
       if "test" in payload:
           # Directly test nova act
           result = _run_browser_task(request=payload.get("test"))
           return result
       elif "prompt" in payload:
           result = graph(payload.get("prompt"))
           return {"result": result.results['start'].result.message}
       else:
           return {"result": "You must provide a `prompt` or `test` key to proceed. ✋"}
   ```
   This function is the entrypoint for our agent, decorated with `@app.entrypoint`. It handles different types of requests.

The file demonstrates how to build an asynchronous agent that can perform background tasks while maintaining a conversation with the user. It leverages AgentCore's task management capabilities and Strands' multi-agent orchestration to create a responsive shopping assistant.

### Creating runtime role

Before deploying our agent, we need to create an IAM role that will be used by AgentCore Runtime. This role will have the necessary permissions to access Amazon Bedrock and other AWS services.

In [ ]:
import sys
import os
import json
import boto3

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

# Navigate up to the utils.py location
utils_dir = os.path.join(current_dir, '..', '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)

from utils import create_agentcore_role

agent_name="async_shopping_assistant"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

### Configure AgentCore Runtime deployment

Now we'll use the AgentCore starter toolkit to configure the deployment of our agent. This will generate a Docker file based on our application code and prepare everything for deployment.

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import time

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="async_shopping_with_strands.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region
)
response

### Launching agent to AgentCore Runtime

Now that we've configured our agent, let's launch it to the AgentCore Runtime. This will create an Amazon ECR repository and deploy our agent to AgentCore Runtime.

In [ ]:
launch_result = agentcore_runtime.launch()
launch_result

### Checking for the AgentCore Runtime Status

Let's check the deployment status of our agent. We'll wait until the status is "READY" before proceeding.

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### Invoking AgentCore Runtime

Now that our agent is deployed and ready, let's invoke it with a product search query. This will demonstrate the asynchronous capabilities of our shopping assistant.

In [ ]:
from IPython.display import Markdown, display

# Invoke the agent with a product search query
invoke_response = agentcore_runtime.invoke({"prompt": "What is the price of Echo dot on amazon.com?"})

# Display the response
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

### Viewing the Asynchronous Shopping Assistant in Action

When you invoke the agent with a product search query, it will launch a background task to search for products while continuing to chat with you. To see this in action:

1. Navigate to the AWS Console and go to the Amazon Bedrock section
![](images/go_to_bedrock_agentcore_console.png)


2. In the Browser Tool tab, you'll see your agent's browser sessions
   ![](images/browser_use_tab.png)

3. You can see the built-in browser sandbox that AgentCore provides
   ![](images/aws_built_in_browser_sandbox.png)

4. Click on "View Live Session" to see the browser in action
   ![](images/click_view_live_session.png)

5. Watch as the agent interacts with the browser to search for products
   ![](images/watch_the_agent_interact_with_browser.png)

You'll notice that the agent is able to continue conversing with you while the product search is running in the background. When the search is complete, you can ask the agent to show you the results.


In [ ]:
# Ask a follow-up question
invoke_response = agentcore_runtime.invoke({"prompt": "While you're searching, can you tell me there are other echo versions I should consider?"})

# Display the response
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

Now, let's check if the search results are ready:

In [ ]:
# Wait a moment to allow the background task to complete
time.sleep(10)

# Ask for the search results
invoke_response = agentcore_runtime.invoke({"prompt": "Do you have the shopping search results now?"})

# Display the response
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

### Using boto3 to Invoke the Agent

You can also use boto3 to invoke your agent programmatically. This is useful for integrating your agent into other applications.

In [ ]:
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "Can you search for wireless headphones?"})
)

if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## Conclusion

In this tutorial, we've built an asynchronous shopping assistant using Strands and Amazon Bedrock AgentCore. Our agent can:

1. Accept product search queries from users
2. Launch background tasks to search for products
3. Continue conversing with the user while searches run
4. Store search results in files when complete
5. Retrieve and present results when the user asks for them

This pattern is particularly valuable for applications where tasks can take time to complete, but users expect responsive interactions. By leveraging AgentCore's asynchronous capabilities and Strands' multi-agent orchestration, we can create agents that provide a much better user experience.

### Key Takeaways

- **Asynchronous agents** provide a better user experience for time-consuming tasks
- **AgentCore Runtime** makes it easy to deploy and scale agents
- **Browser Tool** enables agents to search and extract information from websites
- **Strands** provides a flexible framework for building multi-agent systems
- **File System Integration** allows agents to store and retrieve results

### Next Steps

To further enhance this shopping assistant, you could:

1. Add product comparison capabilities
2. Implement user preferences and personalization
3. Add support for multiple search providers
4. Integrate with e-commerce APIs for real-time pricing and availability
5. Implement memory to remember user preferences across sessions

## Cleanup (Optional)

If you want to clean up the resources created in this tutorial, you can use the following code:

In [ ]:
# Get the agent ID and ECR URI
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
# Initialize clients
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)
iam_client = boto3.client('iam')

# Delete the agent runtime
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

# Delete the ECR repository
response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# Delete the IAM role
policies = iam_client.list_role_policies(
    RoleName=agentcore_iam_role['Role']['RoleName'],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=agentcore_iam_role['Role']['RoleName']
)